In [318]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**Data**

In [319]:
food = pd.read_csv('/content/gdrive/My Drive/Food.csv')
compound = pd.read_csv('/content/gdrive/My Drive/Compound.csv')
nutrient = pd.read_csv('/content/gdrive/My Drive/Nutrient.csv')
content = pd.read_csv('/content/gdrive/My Drive/Content.csv')
health_effect = pd.read_csv('/content/gdrive/My Drive/HealthEffect.csv')
compounds_health_effect = pd.read_csv('/content/gdrive/My Drive/CompoundsHealthEffect.csv')
flavor = pd.read_csv('/content/gdrive/My Drive/Flavor.csv')
compounds_flavor = pd.read_csv('/content/gdrive/My Drive/CompoundsFlavor.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [320]:
food = food.replace('NA',np.nan)
food = food[['id', 'name', 'food_group', 'food_subgroup']]
food = food.dropna()

compound = compound.replace('NA',np.nan)
compound = compound[['id', 'name']]
compound = compound.dropna()

nutrient = nutrient.replace('NA',np.nan)
nutrient = nutrient[['id', 'name']]
nutrient = nutrient.dropna()

content = content.replace('NA',np.nan)
content = content[['source_id', 'source_type', 'food_id', 'orig_content', 'orig_unit']]
content_compound = content[(content['source_type'] == 'Compound')]
content_nutrient = content[(content['source_type'] == 'Nutrient')]
content_compound = content_compound.dropna()
content_nutrient = content_nutrient.dropna()

health_effect = health_effect.replace('NA',np.nan)
health_effect = health_effect[['id', 'name']]
health_effect = health_effect.dropna()

compounds_health_effect = compounds_health_effect.replace('NA',np.nan)
compounds_health_effect = compounds_health_effect[['compound_id', 'health_effect_id']]
compounds_health_effect = compounds_health_effect.dropna()

flavor = flavor.replace('NA',np.nan)
flavor = flavor[['id', 'name']]
flavor = flavor.dropna()

compounds_flavor = compounds_flavor.replace('NA',np.nan)
compounds_flavor = compounds_flavor[['compound_id', 'flavor_id']]
compounds_flavor = compounds_flavor.dropna()

In [321]:
content_compound = content_compound[content_compound.orig_content != 0]
content_nutrient = content_nutrient[content_nutrient.orig_content != 0]
content_compound = content_compound.drop_duplicates(subset=['source_id', 'food_id'])
content_nutrient = content_nutrient.drop_duplicates(subset=['source_id', 'food_id'])

In [322]:
#Food-Compound
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_compound[content_compound['food_id'] == food_id].index
  content_compound.at[index, 'food_name'] = food_name
  content_compound.at[index, 'food_group'] = food_group
  content_compound.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = content_compound[(content_compound['source_id'] == compound_id)].index
  content_compound.at[index, 'source_name'] = compound_name

In [323]:
#Food-Nutrient
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_nutrient[content_nutrient['food_id'] == food_id].index
  content_nutrient.at[index, 'food_name'] = food_name
  content_nutrient.at[index, 'food_group'] = food_group
  content_nutrient.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(nutrient)):
  nutrient_id = nutrient['id'].iloc[i]
  nutrient_name = nutrient[nutrient['id'] == nutrient_id]['name'].iloc[0]
  index = content_nutrient[(content_nutrient['source_id'] == nutrient_id)].index
  content_nutrient.at[index, 'source_name'] = nutrient_name

In [324]:
#Compound-Health Effect
for i in range(len(health_effect)):
  health_effect_id = health_effect['id'].iloc[i]
  health_effect_name = health_effect[health_effect['id'] == health_effect_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['health_effect_id'] == health_effect_id].index
  compounds_health_effect.at[index, 'health_effect_name'] = health_effect_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['compound_id'] == compound_id].index
  compounds_health_effect.at[index, 'compound_name'] = compound_name

In [325]:
#Compound-Flavor
for i in range(len(flavor)):
  flavor_id = flavor['id'].iloc[i]
  flavor_name = flavor[flavor['id'] == flavor_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['flavor_id'] == flavor_id].index
  compounds_flavor.at[index, 'flavor_name'] = flavor_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['compound_id'] == compound_id].index
  compounds_flavor.at[index, 'compound_name'] = compound_name

# **Nutri-Scores**

Preprocessing that needs to be done to compute the Nutri-Scores:

- convert mg/100g to g/100g

- convert kcal/100g to kJ/100g

In [326]:
content_nutrient.orig_unit = content_nutrient.orig_unit.replace('mg/100 g', 'mg/100g')
content_compound.orig_unit = content_compound.orig_unit.replace('mg/100 g', 'mg/100g')

In [327]:
for i in range(len(content_nutrient)):
  if content_nutrient['orig_unit'].iloc[i] == 'mg/100g':
    content_nutrient['orig_content'].iloc[i] /= 1000
    content_nutrient['orig_unit'].iloc[i] = 'g/100g'
  elif content_nutrient['orig_unit'].iloc[i] == 'kcal/100g':
    content_nutrient['orig_content'].iloc[i] *= 4.184
    content_nutrient['orig_unit'].iloc[i] = 'kJ/100g'

for i in range(len(content_compound)):
  if content_compound['orig_unit'].iloc[i] == 'mg/100g':
    content_compound['orig_content'].iloc[i] /= 1000
    content_compound['orig_unit'].iloc[i] = 'g/100g'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [328]:
def scoreEnergy(energy):
  if energy <= 335:
    return 0
  elif energy > 3350:
    return 10
  elif energy > 3015:
    return 9
  elif energy > 2680:
    return 8
  elif energy > 2345:
    return 7
  elif energy > 2010:
    return 6
  elif energy > 1675:
    return 5
  elif energy > 1340:
    return 4
  elif energy > 1005:
    return 3
  elif energy > 670:
    return 2
  elif energy > 335:
    return 1

def scoreSugar(sugar):
  if sugar <= 4.5:
    return 0
  elif sugar > 45:
    return 10
  elif sugar > 40:
    return 9
  elif sugar > 36:
    return 8
  elif sugar > 31:
    return 7
  elif sugar > 27:
    return 6
  elif sugar > 22.5:
    return 5
  elif sugar > 18:
    return 4
  elif sugar > 13.5:
    return 3
  elif sugar > 9:
    return 2
  elif sugar > 4.5:
    return 1

def scoreSaturatedFattyAcis(saturatedFattyAcids):
  if saturatedFattyAcids <= 1:
    return 0
  elif saturatedFattyAcids > 10:
    return 10
  elif saturatedFattyAcids > 9:
    return 9
  elif saturatedFattyAcids > 8:
    return 8
  elif saturatedFattyAcids > 7:
    return 7
  elif saturatedFattyAcids > 6:
    return 6
  elif saturatedFattyAcids > 5:
    return 5
  elif saturatedFattyAcids > 4:
    return 4
  elif saturatedFattyAcids > 3:
    return 3
  elif saturatedFattyAcids > 2:
    return 2
  elif saturatedFattyAcids > 1:
    return 1

def scoreSodium(sodium):
  if sodium <= 90:
    return 0
  elif sodium > 900:
    return 10
  elif sodium > 810:
    return 9
  elif sodium > 720:
    return 8
  elif sodium > 630:
    return 7
  elif sodium > 540:
    return 6
  elif sodium > 450:
    return 5
  elif sodium > 360:
    return 4
  elif sodium > 270:
    return 3
  elif sodium > 180:
    return 2
  elif sodium > 90:
    return 1

def scoreFibers(fibers):
  if fibers <= 0.9:
    return 0
  elif fibers > 4.7:
    return -5
  elif fibers > 3.7:
    return -4
  elif fibers > 2.8:
    return -3
  elif fibers > 1.9:
    return -2
  elif fibers > 0.9:
    return -1

def scoreProteins(proteins):
  if proteins <= 1.6:
    return 0
  elif proteins > 8:
    return -5
  elif proteins > 6.4:
    return -4
  elif proteins > 4.8:
    return -3
  elif proteins > 3.2:
    return -2
  elif proteins > 1.6:
    return -1

def nutriScore(nutri_score):
  if nutri_score >= -15 and nutri_score <= -1:
    return 'A'
  elif nutri_score >= 0 and nutri_score <= 2:
    return 'B'
  elif nutri_score >= 3 and nutri_score <= 10:
    return 'C'
  elif nutri_score >= 11 and nutri_score <= 18:
    return 'D'
  elif nutri_score >= 19 and nutri_score <= 40:
    return 'E'

In [329]:
#add sodium score
target_sodium_id = [3524]
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index = content_compound[content_compound['food_id'] == food_id].index
  df_sodium = content_compound.loc[index]
  df_sodium = df_sodium[df_sodium['source_id'].isin(target_sodium_id)]
  if len(df_sodium) == 0:
    continue
  score = scoreSodium(df_sodium.orig_content.iloc[0])
  content_compound.at[df_sodium.index, 'score'] = score

In [330]:
target_protein_id = [2]
target_carbohydrate_id = [3]
target_fatty_acics_id = [4]
target_fiber_id = [5]
target_energy = [38]
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index = content_nutrient[content_nutrient['food_id'] == food_id].index
  df = content_nutrient.loc[index]

  #add protein score
  df_protein = df[df['source_id'].isin(target_protein_id)]
  if len(df_protein) == 0:
    continue
  score = scoreProteins(df_protein.orig_content.iloc[0])
  content_nutrient.at[df_protein.index, 'score'] = score

  #add carbohydrate score
  df_carbohydrate = df[df['source_id'].isin(target_carbohydrate_id)]
  if len(df_carbohydrate) == 0:
    continue
  score = scoreSugar(df_carbohydrate.orig_content.iloc[0])
  content_nutrient.at[df_carbohydrate.index, 'score'] = score

  #add fatty acids score
  df_fatty_acids = df[df['source_id'].isin(target_fatty_acics_id)]
  if len(df_fatty_acids) == 0:
    continue
  score = scoreSaturatedFattyAcis(df_fatty_acids.orig_content.iloc[0])
  content_nutrient.at[df_fatty_acids.index, 'score'] = score

  #add fiber score
  df_fiber = df[df['source_id'].isin(target_fiber_id)]
  if len(df_fiber) == 0:
    continue
  score = scoreFibers(df_fiber.orig_content.iloc[0])
  content_nutrient.at[df_fiber.index, 'score'] = score

  #add energy score
  df_energy = df[df['source_id'].isin(target_energy)]
  if len(df_energy) == 0:
    continue
  score = scoreEnergy(df_energy.orig_content.iloc[0])
  content_nutrient.at[df_energy.index, 'score'] = score

In [331]:
#add Nutri-Score
target_nutrient_id = [2,3,4,5,38]
temp_nutrient = content_nutrient[content_nutrient['source_id'].isin(target_nutrient_id)]

target_compound_id = [3524]
temp_compound = content_compound[content_compound['source_id'].isin(target_compound_id)]

for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index_nutrient = content_nutrient[content_nutrient['food_id'] == food_id].index
  index_compound = content_compound[content_compound['food_id'] == food_id].index

  temp_index_nutrient = temp_nutrient[temp_nutrient['food_id'] == food_id].index
  temp_index_compound = temp_compound[temp_compound['food_id'] == food_id].index
  df_nutrient = temp_nutrient.loc[temp_index_nutrient]
  df_compound = temp_compound.loc[temp_index_compound]
  df = pd.concat([df_nutrient, df_compound])
  nutri_score = df['score'].sum(axis = 0)
  nutri_score_category = nutriScore(nutri_score)

  content_nutrient.at[index_nutrient, 'nutri_score'] = nutri_score
  content_compound.at[index_compound, 'nutri_score'] = nutri_score
  content_nutrient.at[index_nutrient, 'nutri_score_category'] = nutri_score_category
  content_compound.at[index_compound, 'nutri_score_category'] = nutri_score_category

In [332]:
content_compound.to_csv('content_compound.csv')
content_nutrient.to_csv('content_nutrient.csv')
compounds_health_effect.to_csv('compounds_health_effect.csv')
compounds_flavor.to_csv('compounds_flavor.csv')

In [334]:
from google.colab import files
files.download('content_compound.csv')
files.download('content_nutrient.csv')
files.download('compounds_health_effect.csv')
files.download('compounds_flavor.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>